In [1]:
%cd C:\Users\yukir\Documents\GitHub\Text_Mining\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Korea_medicine\LDA

C:\Users\yukir\Documents\GitHub\Text_Mining\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3036186368008501389
]
tf 2.5.3
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


C:\Users\yukir\Documents\Monicas_workspace\Korea_medicine\LDA


In [2]:
import openpyxl
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv

### 띄어쓰기

In [50]:
%cd C:\Users\yukir\Documents\Monicas_workspace\Derma

C:\Users\yukir\Documents\Monicas_workspace\Derma


In [51]:
reviews_df = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\[0426]derma_data_touse.xlsx", index_col=0)
reviews_df.head()


,h,reviews,stars,review_date,do,si,ro
0,고운나라피부과의원,두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,"3월 15일, 2022년",경남,김해시,내외중앙로
1,고운나라피부과의원,대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,"12월 20일, 2021년",경남,김해시,내외중앙로
2,고운나라피부과의원,여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,"8월 11일, 2021년",경남,김해시,내외중앙로
3,고운나라피부과의원,문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,"6월 21일, 2021년",경남,김해시,내외중앙로
4,고운나라피부과의원,원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,"6월 7일, 2021년",경남,김해시,내외중앙로


In [52]:
reviews_df.shape

(39216, 7)

In [53]:
data = reviews_df.copy()
# data = data[20000:]
print(data.shape)

(39216, 7)


In [14]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

In [13]:
def stemming (text):
    morphs_list = []
    one_words = []
    result = []

    pos = okt.pos(text, join = True)
    # print(pos)
    for j in pos:
        if j.split('/')[1] == 'Noun':
            j = j.split('/')[0]
            morphs_list.append(j)
                    
        elif j.split('/')[1] =='Adjective':
            k = okt.morphs(j,  stem= True)
            k = k[0]
            morphs_list.append(k)
        elif j.split('/')[1] =='Verb':
            v = okt.morphs(j,  stem= True)
            v = v[0]
            morphs_list.append(v)

    for i in morphs_list:
        if len(i) != 1:
            one_words.append(i)
    

    for i in one_words:
        if i not in stopwords:
            result.append(i)

    return result

In [12]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 
    
def get_nouns(tokenizer, sentence): 
    """ 단어의 길이가 2이상인 일반명사(NNG),
     고유명사(NNP), 외국어(SL)만을 반환한다. 
     :param tokenizer: 
     :param sentence: :return: """ 
    tagged = tokenizer.nouns(sentence)
    nouns = [s for s in tagged if len(s)>1] 
     
    return nouns 
     
     
def tokenize(df): 
    tokenizer = okt
    processed_data = [] 
    for sent in tqdm(df['cleaned_reviews']):
        sentence = clean_text(sent.replace('\n', '').strip()) 
        processed_data.append(get_nouns(tokenizer, sentence)) 
        
    return processed_data
    
    
def save_processed_data(processed_data): 
    """ 토큰 분리한 데이터를 csv로 저장 :param processed_data: :return: """ 
    
    with open('lsa_token_pos.csv', 'w', newline='', encoding='utf-8') as f: 
        writer = csv.writer(f) 
        for data in processed_data:
             writer.writerow(data)

In [56]:
review_list = data.reviews.tolist()

In [57]:
from hanspell import spell_checker

In [58]:
from tqdm import tqdm

In [59]:
cleaned_reviews= []

for i in tqdm (review_list):
    try: 
        i = i.replace(' ','')
        i = i.replace('ㆍ','')
        i = clean_text(i)
        i = i.replace('(', ' ')
        i = i.replace(')',' ')
        i = spell_checker.check(i)
        i = i.checked
        cleaned_reviews.append(i)
    except:
        cleaned_reviews.append('에러')
    # print(i)

100%|██████████| 39216/39216 [4:09:03<00:00,  2.62it/s]   


In [60]:
data.head()

,h,reviews,stars,review_date,do,si,ro
0,고운나라피부과의원,두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,"3월 15일, 2022년",경남,김해시,내외중앙로
1,고운나라피부과의원,대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,"12월 20일, 2021년",경남,김해시,내외중앙로
2,고운나라피부과의원,여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,"8월 11일, 2021년",경남,김해시,내외중앙로
3,고운나라피부과의원,문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,"6월 21일, 2021년",경남,김해시,내외중앙로
4,고운나라피부과의원,원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,"6월 7일, 2021년",경남,김해시,내외중앙로


In [61]:
data['cleaned_reviews'] = cleaned_reviews

### cleansing

In [62]:
%cd C:\Users\yukir\Documents\Monicas_workspace\Derma

C:\Users\yukir\Documents\Monicas_workspace\Derma


In [63]:
from tqdm import tqdm

In [64]:
data_  =[]

for i in tqdm (data['cleaned_reviews'].tolist()):
    try: 
        if ' 한 줄 평 ' in i:
            i = i.replace(' 한 줄 평','')
            data_.append(i)
        else:
            data_.append(i)
    except:
        data_.append('리뷰없음')
            

100%|██████████| 39216/39216 [00:00<00:00, 1452382.99it/s]


In [65]:
data['cleaned_reviews'] = data_
data.head()

,h,reviews,stars,review_date,do,si,ro,cleaned_reviews
0,고운나라피부과의원,두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,"3월 15일, 2022년",경남,김해시,내외중앙로,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...
1,고운나라피부과의원,대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,"12월 20일, 2021년",경남,김해시,내외중앙로,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...
2,고운나라피부과의원,여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,"8월 11일, 2021년",경남,김해시,내외중앙로,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...
3,고운나라피부과의원,문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,"6월 21일, 2021년",경남,김해시,내외중앙로,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...
4,고운나라피부과의원,원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,"6월 7일, 2021년",경남,김해시,내외중앙로,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...


In [66]:
from nltk.corpus import stopwords

In [67]:
def make_noun(x):
    x = okt.nouns(x)
    return x

In [68]:
data['reviews_stem'] = data['cleaned_reviews'].apply(make_noun)
data.head() 

,h,reviews,stars,review_date,do,si,ro,cleaned_reviews,reviews_stem
0,고운나라피부과의원,두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,"3월 15일, 2022년",경남,김해시,내외중앙로,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...,"[두피, 가려움증, 원한, 병원, 진료, 두피, 현미경, 사진, 무료, 상담, 제,..."
1,고운나라피부과의원,대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,"12월 20일, 2021년",경남,김해시,내외중앙로,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...,"[대상포진, 진료, 주, 이상, 약, 주사, 진통, 대상포진, 통증, 후유증, 남아..."
2,고운나라피부과의원,여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,"8월 11일, 2021년",경남,김해시,내외중앙로,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...,"[여드름, 고민, 방문, 피부, 전문의, 증상, 이유, 대해, 설명, 지인, 추천,..."
3,고운나라피부과의원,문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,"6월 21일, 2021년",경남,김해시,내외중앙로,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...,"[문신, 제거, 문신, 곳일, 부분, 여러, 군데, 워낙, 가격, 천차만별, 치료,..."
4,고운나라피부과의원,원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,"6월 7일, 2021년",경남,김해시,내외중앙로,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...,"[원장, 설명, 병원, 김해, 곳, 유가, 직원, 전부, 맘, 처방, 약, 하루, ..."


In [69]:
data.to_excel('[0427]data_derma.xlsx')

In [5]:
%cd C:\Users\yukir\Documents\Monicas_workspace\Derma\TF_IDF

C:\Users\yukir\Documents\Monicas_workspace\Derma\TF_IDF


In [6]:
data = pd.read_excel('../[0427]data_derma.xlsx', index_col=0)
data.head()

,h,reviews,stars,review_date,do,si,ro,cleaned_reviews,reviews_stem
0,고운나라피부과의원,두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,"3월 15일, 2022년",경남,김해시,내외중앙로,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...,"['두피', '가려움증', '원한', '병원', '진료', '두피', '현미경', ..."
1,고운나라피부과의원,대상포진으로 진료 받았어요 . 2주 이상 치료 받고 약 먹고 주사 맞으니까 진통이 ...,10,"12월 20일, 2021년",경남,김해시,내외중앙로,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...,"['대상포진', '진료', '주', '이상', '약', '주사', '진통', '대상..."
2,고운나라피부과의원,여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,"8월 11일, 2021년",경남,김해시,내외중앙로,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...,"['여드름', '고민', '방문', '피부', '전문의', '증상', '이유', '..."
3,고운나라피부과의원,문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,"6월 21일, 2021년",경남,김해시,내외중앙로,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...,"['문신', '제거', '문신', '곳일', '부분', '여러', '군데', '워낙..."
4,고운나라피부과의원,원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,"6월 7일, 2021년",경남,김해시,내외중앙로,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...,"['원장', '설명', '병원', '김해', '곳', '유가', '직원', '전부'..."


### TF_IDF

In [9]:
print(len(data))
data = data.dropna()
print(len(data))
data = data.drop_duplicates('cleaned_reviews')
print(len(data))

39216
38857
30984


In [11]:
for sent in tqdm(data['cleaned_reviews']):
    try:
        sentence = clean_text(sent.replace('\n', '').strip())
        # print(sentence)
    except:
        # print(sent)
        pass

100%|██████████| 30984/30984 [00:00<00:00, 1290965.32it/s]


In [15]:
## 오래걸리니까 조심

if __name__ == '__main__':
     # df = pd.read_csv('negative.csv', index_col = 0)
     processed_data = tokenize(data) # description 부분을 토크나이징 한다. 
     
     # 토큰 분리한 데이터를 저장 
     save_processed_data(processed_data)


100%|██████████| 30984/30984 [10:23<00:00, 49.67it/s] 


In [19]:
detokenized_doc = []
for i in range(len(processed_data)):
    t = ' '.join(processed_data[i])
    detokenized_doc.append(t)

data['detokenized_doc'] = detokenized_doc # 다시 text['headline_text']에 재저장

data['detokenized_doc'][:5]

0      두피 가려움증 원한 병원 진료 두피 현미경 사진 무료 상담 두피 샴푸 하나 샴푸 방법
1           대상포진 진료 이상 주사 진통 대상포진 통증 후유증 남아 부위 계속 예방접종
2    여드름 고민 방문 피부 전문의 증상 이유 대해 설명 지인 추천 의사 선생님 진료 상...
3    문신 제거 문신 곳일 부분 여러 군데 워낙 가격 천차만별 치료 합리 가격 부분 레이...
4                     원장 설명 병원 김해 유가 직원 전부 처방 하루 피부 아주
Name: detokenized_doc, dtype: object

In [20]:
from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=stopwords_kor,             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[가-힣0-9]{1,}',  # num chars > 3
                             max_features=2000,             # max number of uniq words
                            ) 
data_vectorized = vectorizer.fit_transform(data['detokenized_doc'])
data_vectorized.shape # TF-IDF 행렬의 크기 확인

(30984, 2000)

In [28]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
dist = np.sum(data_vectorized, axis=0)
    
df_freq = pd.DataFrame(dist, columns=vocab)
print(df_freq.shape)

2000
(1, 2000)


In [31]:
df_freq.T.sort_values(by=0, ascending=False).head(100)
df_freq_T = df_freq.T.reset_index()
df_freq_T.columns = ["words", "freq"]
df_freq_T.tail()

,words,freq
1995,흑자,6.174916
1996,흔적,20.093027
1997,흡입,23.450061
1998,희망,6.985679
1999,힐러,14.921240


In [32]:
print(df_freq_T.shape)
df_use = df_freq_T.drop_duplicates()
print(df_use.shape)

(2000, 2)
(2000, 2)


In [59]:
df_use = df_use.sort_values(by="freq", ascending=False)
df_use.head(100)

,words,freq
1621,진료,2175.409307
687,병원,2039.753294
1869,피부,1919.997208
645,방문,1617.243045
1721,치료,1600.922409
...,...,...
1760,크림,196.374847
821,상세,195.775792
71,개선,193.947347
707,보통,193.650820


In [60]:
df_use.to_excel('derma_TFIDF_to_SERVQUAL.xlsx')